In [12]:
import numpy as np
from pykml.factory import KML_ElementMaker as KML
from pykml.parser import Schema
from pykml import parser
from pykml.factory import write_python_script_for_kml_document
from lxml import etree
import pandas as pd

In [ ]:
data = pd.read_csv("server_data/site_node_info.csv", sep=",", header=0)
node_id = data["node_id"].as_matrix()
site_name = data["site_id"].as_matrix()
lat = data["lat"].as_matrix()
lon = data["long"].as_matrix()
print node_id
print site_name
print lat
print lon

pm1_loc = str(lon[0]) + "," + str(lat[0])
pm1 = KML.Placemark(
          KML.name(site_name[0] + "_" + str(node_id[0])),
          KML.Point(
            KML.coordinates(pm1_loc)
          )
        )
fld = KML.Folder(pm1)
for i in range(1, len(node_id)):
    pm_loc = str(lon[i]) + "," + str(lat[i])
    pm = KML.Placemark(
             KML.name(site_name[i] + "_" + str(node_id[i])),
             KML.Point(
                KML.coordinates(pm_loc)
        )
    )
    fld.append(pm)
print etree.tostring(fld, pretty_print=True)
outfile = file('AR_sensors.kml','w')
outfile.write(etree.tostring(fld, pretty_print=True))
assert Schema('kml22gx.xsd').validate(fld)

In [14]:
with open("doc.kml") as f:
    doc = parser.parse(f).getroot()
print(doc)
script = write_python_script_for_kml_document(doc)
print(script)


from lxml import etree
from pykml.factory import KML_ElementMaker as KML
from pykml.factory import ATOM_ElementMaker as ATOM
from pykml.factory import GX_ElementMaker as GX

doc = KML.kml(
  KML.Document(
    KML.name('Experimental Forest Boundary'),
    KML.Snippet(),
    KML.Folder(
      KML.name('Experimental Forest Boundary'),
      KML.Snippet(),
      KML.Placemark(
        KML.name('Experimental Forest'),
        KML.Snippet(),
        KML.description(
'<html xmlns:fo="http://www.w3.org/1999/XSL/Format" xmlns:msxsl="urn:schemas-microsoft-com:xslt">'
'<head>'
'<META http-equiv="Content-Type" content="text/html">'
'<meta http-equiv="content-type" content="text/html; charset=UTF-8">'
'</head>'
'<body style="margin:0px 0px 0px 0px;overflow:auto;background:#FFFFFF;">'
'<table style="font-family:Arial,Verdana,Times;font-size:12px;text-align:left;width:100%;border-collapse:collapse;padding:3px 3px 3px 3px">'
'<tr style="text-align:center;font-weight:bold;background:#9CBCE2">'
'<td>Ex

In [16]:
import simplekml
import mysql.connector
from mysql.connector import errorcode
import numpy as np

site_names_query = "SELECT site_name, site_id FROM sites"
site_node_info_query = "SELECT lat, lon FROM motes WHERE site_id = %s"
kml = simplekml.Kml()
cnx = mysql.connector.connect(user = "root", password = "root", database = "ar_data")
cursor = cnx.cursor()
cursor.execute(site_names_query)
site_info = cursor.fetchall()
print(site_info)
for temp_site_info in site_info:
    site_name = temp_site_info[0]
    site_id = temp_site_info[1]
    cursor.execute(site_node_info_query, (site_id, ))
    site_lat_lon = cursor.fetchall()
    lat = []
    lon = []
    for lat_lon in site_lat_lon:
        lat.append(lat_lon[0])
        lon.append(lat_lon[1])
    lat_min = min(lat) - 0.001
    lat_max = max(lat) + 0.001
    lon_min = min(lon) - 0.001
    lon_max = max(lon) + 0.001
    print(lat_min, lat_max, lon_min, lon_max)
    pol = kml.newpolygon(name = site_name)
    pol.outerboundaryis = [(lon_min, lat_min), (lon_min, lat_max), (lon_max, lat_max), (lon_max, lat_min), (lon_min, lat_min)]
    pol.style.linestyle.color = simplekml.Color.black
    pol.style.linestyle.width = 1
    pol.style.polystyle.color = simplekml.Color.blue
pol = kml.newpolygon(name = "Lost_Corner")
lat_min = 39.014914 - 0.001
lat_max = 39.018996 + 0.001
lon_min = -120.217043 - 0.001
lon_max = -120.21246 + 0.001
pol.outerboundaryis = [(lon_min, lat_min), (lon_min, lat_max), (lon_max, lat_max), (lon_max, lat_min), (lon_min, lat_min)]
pol.style.linestyle.color = simplekml.Color.black
pol.style.linestyle.width = 1
pol.style.polystyle.color = simplekml.Color.blue

pol = kml.newpolygon(name = "Pine_Crest")
pol.outerboundaryis = [(-119.9760229108838,38.18101594894384), (-119.99441048825,38.1810382941641), 
                       (-120.0010430323597,38.18101893525246), (-120.0036689109387,38.18100499102879), 
                       (-120.0036686499068,38.17739211016583), (-120.008314748439,38.17737561631331), 
                       (-120.008299347781,38.16436524595073), (-120.0037107102265,38.16439223010647),
                       (-120.0037264060437,38.15899583210247), (-120.0010446957601,38.1589993522845), 
                       (-119.9943909871472,38.15902721101466), (-119.9944012893485,38.16655054251656), 
                       (-119.985238915283,38.16651353875074), (-119.9852879903711,38.17196563028903), 
                       (-119.9760091194904,38.17196994566627), (-119.9760071562137,38.17367271193644), 
                       (-119.9668185770823,38.17369664300085), (-119.9668580776497,38.1809954325157), 
                       (-119.9760229108838,38.18101594894384)]
pol.style.linestyle.color = simplekml.Color.black
pol.style.linestyle.width = 1
pol.style.polystyle.color = simplekml.Color.blue

cursor.close()
cnx.close()
kml.save("AR_polygon.kml")
# kml = simplekml.Kml()

[(u'Alpha', 1), (u'Bear_Trap', 2), (u'Caples_Lk', 3), (u'DollyRice', 4), (u'Duncan_Pk', 5), (u'Echo_Pk', 6), (u'Mt_Lincoln', 7), (u'Onion_Ck', 8), (u'Owens_Camp', 9), (u'Robbs_Saddle', 10), (u'Schneiders', 11), (u'Talbot_Camp', 12), (u'Van_Vleck', 13)]
(38.800453000000005, 38.809998, -120.219002, -120.208862)
(39.089000000000006, 39.098492, -120.582429, -120.56998399999999)
(38.708286, 38.713437, -120.04481600000001, -120.039321)
(39.146221000000004, 39.150878999999996, -120.373704, -120.368102)
(39.14735, 39.15651, -120.514718, -120.505516)
(38.846118000000004, 38.856282, -120.080865, -120.068542)
(39.28226, 39.288529, -120.329514, -120.31974699999999)
(39.272258, 39.279838999999996, -120.36261, -120.352584)
(38.733982000000005, 38.738476, -120.244179, -120.238838)
(38.910171000000005, 38.915574, -120.38245400000001, -120.375389)
(38.741638, 38.747955999999995, -120.073548, -120.06255999999999)
(39.188823, 39.19262, -120.379075, -120.374237)
(38.94004, 38.944398, -120.31250800000001, 